In [1]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb
from model import SAGE, GAT, get_eigen
from train_eval import train_ft_embed

Using backend: pytorch


This notebook shows the performance of GAT on the OGBN-products graph with trainable embeddings on the nodes. We use scaled eigenvectors to initialize the node embeddings. The logic here is that if eigenvectors can serve as positional node embeddings, fine-tuning the node embeddings should give us even better performance. In this experiment, we tune the node embeddings with a much smaller learning rate than the one for the GAT model.

In [4]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))), dtype=th.float32)
labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 1024,
    'num_workers': 0,
    'num_hidden': 256,
    'num_layers': 3,
    'num_heads': 2,
    'dropout': 0.5,
    'lr': 0.003,
    'sparse_lr': 0.0003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:6')

# Pack data
data = g, nfeat, labels
# Define model
model = GAT(in_feats, hyperparams['num_hidden'], num_labels,
             hyperparams['num_layers'], hyperparams['num_heads'], F.relu, hyperparams['dropout'])
train_ft_embed(model, data, hyperparams, device, 1)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products
embedding is in  cuda:6


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 3.8490 | Train Acc 0.0137 | Speed (samples/sec) nan | GPU 5588.5 MB
Epoch 00000 | Step 00020 | Loss 1.8869 | Train Acc 0.4873 | Speed (samples/sec) 315.0262 | GPU 8123.7 MB
Epoch 00000 | Step 00040 | Loss 1.2861 | Train Acc 0.6924 | Speed (samples/sec) 313.2386 | GPU 8138.2 MB
Epoch 00000 | Step 00060 | Loss 0.8674 | Train Acc 0.7920 | Speed (samples/sec) 314.4934 | GPU 8138.2 MB
Epoch 00000 | Step 00080 | Loss 0.6539 | Train Acc 0.8496 | Speed (samples/sec) 319.1254 | GPU 8138.2 MB
Epoch 00000 | Step 00100 | Loss 0.5180 | Train Acc 0.8672 | Speed (samples/sec) 318.0370 | GPU 8138.2 MB
Epoch 00000 | Step 00120 | Loss 0.4248 | Train Acc 0.8926 | Speed (samples/sec) 315.9198 | GPU 8138.2 MB
Epoch 00000 | Step 00140 | Loss 0.4437 | Train Acc 0.8789 | Speed (samples/sec) 318.3943 | GPU 8138.2 MB
Epoch 00000 | Step 00160 | Loss 0.4237 | Train Acc 0.8887 | Speed (samples/sec) 323.3136 | GPU 8138.2 MB
Epoch 00000 | Step 00180 | Loss 0.3965 | Train Acc 0.9004 | 

100%|██████████| 245/245 [01:48<00:00,  2.26it/s]


Val acc 0.9116, Test Acc: 0.7817
Best val acc 0.9116, best test acc: 0.7817
Epoch 00002 | Step 00000 | Loss 0.3680 | Train Acc 0.8975 | Speed (samples/sec) 337.7296 | GPU 8149.1 MB
Epoch 00002 | Step 00020 | Loss 0.3212 | Train Acc 0.9160 | Speed (samples/sec) 339.2276 | GPU 8149.1 MB
Epoch 00002 | Step 00040 | Loss 0.3693 | Train Acc 0.9072 | Speed (samples/sec) 339.4118 | GPU 8149.1 MB
Epoch 00002 | Step 00060 | Loss 0.3558 | Train Acc 0.9043 | Speed (samples/sec) 339.8712 | GPU 8149.1 MB
Epoch 00002 | Step 00080 | Loss 0.3283 | Train Acc 0.8975 | Speed (samples/sec) 341.1143 | GPU 8149.1 MB
Epoch 00002 | Step 00100 | Loss 0.4010 | Train Acc 0.8994 | Speed (samples/sec) 341.1471 | GPU 8159.8 MB
Epoch 00002 | Step 00120 | Loss 0.3119 | Train Acc 0.9092 | Speed (samples/sec) 341.1300 | GPU 8159.8 MB
Epoch 00002 | Step 00140 | Loss 0.2889 | Train Acc 0.9072 | Speed (samples/sec) 341.4259 | GPU 8159.8 MB
Epoch 00002 | Step 00160 | Loss 0.3774 | Train Acc 0.8896 | Speed (samples/sec) 342.

...

Val acc 0.9213, Test Acc: 0.7917
Epoch 00018 | Step 00000 | Loss 0.2835 | Train Acc 0.9170 | Speed (samples/sec) 357.5810 | GPU 8159.8 MB
Epoch 00018 | Step 00020 | Loss 0.2417 | Train Acc 0.9258 | Speed (samples/sec) 357.9221 | GPU 8159.8 MB
Epoch 00018 | Step 00040 | Loss 0.2661 | Train Acc 0.9141 | Speed (samples/sec) 358.3325 | GPU 8159.8 MB
Epoch 00018 | Step 00060 | Loss 0.2982 | Train Acc 0.9062 | Speed (samples/sec) 358.7283 | GPU 8159.8 MB
Epoch 00018 | Step 00080 | Loss 0.3204 | Train Acc 0.9072 | Speed (samples/sec) 359.0490 | GPU 8159.8 MB
Epoch 00018 | Step 00100 | Loss 0.2456 | Train Acc 0.9219 | Speed (samples/sec) 359.3945 | GPU 8159.8 MB
Epoch 00018 | Step 00120 | Loss 0.2646 | Train Acc 0.9121 | Speed (samples/sec) 359.7576 | GPU 8159.8 MB
Epoch 00018 | Step 00140 | Loss 0.2129 | Train Acc 0.9277 | Speed (samples/sec) 360.2033 | GPU 8159.8 MB
Epoch 00018 | Step 00160 | Loss 0.2638 | Train Acc 0.9238 | Speed (samples/sec) 360.5990 | GPU 8159.8 MB
Epoch 00018 | Step 001

100%|██████████| 245/245 [01:33<00:00,  2.63it/s]


Val acc 0.9222, Test Acc: 0.7939
Epoch 00019 | Step 00000 | Loss 0.2356 | Train Acc 0.9238 | Speed (samples/sec) 361.1536 | GPU 8159.8 MB
Epoch 00019 | Step 00020 | Loss 0.2486 | Train Acc 0.9141 | Speed (samples/sec) 361.4866 | GPU 8159.8 MB
Epoch 00019 | Step 00040 | Loss 0.2841 | Train Acc 0.9131 | Speed (samples/sec) 361.8552 | GPU 8159.8 MB
Epoch 00019 | Step 00060 | Loss 0.2481 | Train Acc 0.9219 | Speed (samples/sec) 362.2297 | GPU 8159.8 MB
Epoch 00019 | Step 00080 | Loss 0.2452 | Train Acc 0.9170 | Speed (samples/sec) 362.5385 | GPU 8159.8 MB
Epoch 00019 | Step 00100 | Loss 0.1988 | Train Acc 0.9336 | Speed (samples/sec) 362.8643 | GPU 8159.8 MB
Epoch 00019 | Step 00120 | Loss 0.2393 | Train Acc 0.9219 | Speed (samples/sec) 363.2672 | GPU 8159.8 MB
Epoch 00019 | Step 00140 | Loss 0.2558 | Train Acc 0.9307 | Speed (samples/sec) 363.6041 | GPU 8159.8 MB
Epoch 00019 | Step 00160 | Loss 0.2388 | Train Acc 0.9277 | Speed (samples/sec) 364.0545 | GPU 8159.8 MB
Epoch 00019 | Step 001

100%|██████████| 245/245 [01:32<00:00,  2.65it/s]


Val acc 0.9194, Test Acc: 0.7858
Avg epoch time: 687.2379230499267
Best val acc 0.9223, best test acc: 0.7885
